#### LOAD LIBRARIES

In [39]:
# Pulizia
from IPython import get_ipython
get_ipython().magic('clear')
get_ipython().magic('reset -f')

# Import libraries:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg
import itertools
import time
import sys
import json

# Import user defined libraries:
from FreeFEM import FFmatrix_fread, FFvector_fread

/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_1682/4168349740.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('clear')
/var/folders/gs/p2p59ns16hlgpqv_027665gm0000gn/T/ipykernel_1682/4168349740.py:4: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -f')


#### DEFINE PROBLEM PARAMETERS

In [47]:
# Lettura dei parametri di riferimento dal file JSON
with open("data/Reference_parameters.json", "r") as json_file:
    parameters = json.load(json_file)
N_AVOG = parameters["N_AVOG"]  
N_BOLT = parameters["N_BOLT"]  
FISSION_ENERGY = parameters["FISSION_ENERGY"]                       
FISSION_RATE = parameters["FISSION_RATE"]                   
RADIUS = parameters["RADIUS"]
LENGTH = parameters["LENGTH"]
FISSION_YIELD = parameters["FISSION_YIELD"]                 
FUEL_THERMAL_CONDUCTIVITY = parameters["FUEL_THERMAL_CONDUCTIVITY"]
T_BC = parameters["T_BC"]  
C_BC = parameters["C_BC"]  

SOURCE_C = FISSION_RATE * FISSION_YIELD         #[atm / (m^3 s)]
GAMMA_T = (FISSION_RATE * FISSION_ENERGY * LENGTH**2)/(FUEL_THERMAL_CONDUCTIVITY)

# Define reference temperature, concentration and linear diffusion parameters:
T_IC  = T_BC 
C_IC = C_BC

# Define final time, time increment:
TIME_FINAL = 1.00E+07  # [s]
TIME_DELTA = 1.00E+04  # [s]

# Compute number of iterations:
N_ITER = int(TIME_FINAL / TIME_DELTA)  # [-]

# Define function evaluating the concentration diffusion coefficient at given z coordinated provided the problem parameters:
def ALPHA_C(ZZ, T_BC=T_BC, GAMMA_T=GAMMA_T):
    return 2.949513e-13 * np.exp(-20487.36244 / (T_BC + GAMMA_T * (1 - ZZ**2) / 2));

#### IDENTIFY DIRICHLET DOFS AND INTERNAL DOFS

In [43]:
# Load coordinates:
coordinates_Px = FFvector_fread('mesh_utilities/vv_cc_Px.btxt')
coordinates_Pq = FFvector_fread('mesh_utilities/vv_cc_Pq.btxt')

# Identify degrees of freedom of the Px and Pq spaces:
sFO_Px = coordinates_Px.shape[0]
sFO_Pq = coordinates_Pq.shape[0]

# Define vector of all Px indeces:
mask_all = np.arange(sFO_Px, dtype=int)

# Identify indeces of the top, bottom and middle boundaries in Px:
mask_inf_bc = mask_all[np.isclose([coordinates_Px[ii, 2] for ii in range(sFO_Px)], np.zeros((sFO_Px)))]
mask_sup_bc = mask_all[np.isclose([coordinates_Px[ii, 2] for ii in range(sFO_Px)], np.ones((sFO_Px)))]
mask_mid_bc = mask_all[np.isclose([np.linalg.norm(coordinates_Px[ii, :2]) for ii in range(sFO_Px)], np.ones((sFO_Px)))]

# Identify indeces of the Dirichlet b.c. for the temperature and concentration field:
mask_bc_T = np.fromiter(set(mask_sup_bc), int)
mask_bc_C = np.fromiter(set(mask_sup_bc) | set(mask_mid_bc) | set(mask_inf_bc), int)

# Identify indeces of the interior for the temperature and concentration field:
mask_in_T = [ii for ii in mask_all if ii not in mask_bc_T]
mask_in_C = [ii for ii in mask_all if ii not in mask_bc_C]

#### IMPORT FINITE ELEMENT ESSENTIALS

In [44]:
# Import mass matrices on the Px and on the Pq space:
mass_Px = FFmatrix_fread('mesh_utilities/ww_mm_Px.btxt')
mass_Pq = FFmatrix_fread('mesh_utilities/ww_mm_Pq.btxt')

# Compute volume of the domain:
volume = mass_Px.dot(np.ones(sFO_Px)).dot(np.ones(sFO_Px))

# Extract integration weights and diagonal matrix with their inverse:
weights_Pq = mass_Pq.diagonal()
project_Pq = sparse.diags(np.reciprocal(weights_Pq))

# Import map from Px to Pq and to the dx, dy, dz derivative evaluated in Pq:
PxtoPquu_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_uu_Px_Pq.btxt')[:, mask_in_C])
PxtoPqdx_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_dx_Px_Pq.btxt')[:, mask_in_C])
PxtoPqdy_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_dy_Px_Pq.btxt')[:, mask_in_C])
PxtoPqdz_C = project_Pq.dot(FFmatrix_fread('mesh_utilities/ww_dz_Px_Pq.btxt')[:, mask_in_C])

# Assemble mass matrix and projected forcing term:
forc_C = PxtoPquu_C.T.dot(weights_Pq[:, None])
mass_C = PxtoPquu_C.T.dot(PxtoPquu_C.multiply(weights_Pq[:, None]))
inte_C = mass_Px.dot(np.ones((sFO_Px)))[mask_in_C] / volume

# Assemble a function assembling the stiffness matrix at a given time:
stiff_C = PxtoPqdx_C.T.dot(PxtoPqdx_C.multiply(ALPHA_C(coordinates_Pq[:, 2:3]) / RADIUS**2 * weights_Pq[:, None])) + \
          PxtoPqdy_C.T.dot(PxtoPqdy_C.multiply(ALPHA_C(coordinates_Pq[:, 2:3]) / RADIUS**2 * weights_Pq[:, None])) + \
          PxtoPqdz_C.T.dot(PxtoPqdz_C.multiply(ALPHA_C(coordinates_Pq[:, 2:3]) / LENGTH**2 * weights_Pq[:, None]))

#### IMPLEMENT FULLY IMPLICID FORWARD EULER FOR THE CONCENTRATION ONLY

In [ ]:
# Initialise temperature and concentration solution containing the deviation from the initial condition:
sol_new_C = np.zeros((sFO_Px, N_ITER+1)) 
average_C = np.zeros(N_ITER)             

# Crea o sovrascrivi il file CSV all'inizio
with open('data/Solution.csv', 'w') as f:
    f.write('Time (s),Average dC (atm/m^3)\n')  # Intestazione delle colonne

for ii in range(N_ITER):
    cur_time = ii * TIME_DELTA

    # Print current iteration and average temperature and concentration:
    print('Current time:', cur_time, 's')
    average_C[ii] = inte_C @ sol_new_C[mask_in_C, ii]
    print('Average dC:', average_C[ii], 'atm/m^3\n')

    # Salva i dati correnti nel file CSV
    with open('data/Solution.csv', 'a') as f:
        f.write(f'{cur_time},{average_C[ii]}\n')

    # Assemble current left-hand-side and right-hand-side
    cur_lhs = mass_C + TIME_DELTA * stiff_C
    cur_rhs = mass_C.dot(sol_new_C[mask_in_C, ii:ii+1]) + TIME_DELTA * (SOURCE_C * forc_C)

    # Compute perturbation from the initial temperature and concentration in the interior of the domain:
    sol_new_C[mask_in_C, ii+1], _ = linalg.bicgstab(cur_lhs, cur_rhs)
    #sol_new_C[mask_in_C, ii+1] = linalg.spsolve(cur_lhs, cur_rhs)
    #sol_new_C[mask_in_C, ii+1], _ = linalg.gmres(cur_lhs, cur_rhs)

# Print final time and final average temperature and concentration:
final_time = N_ITER * TIME_DELTA
final_average_dC = mass_Px.dot(sol_new_C[:, -1]).dot(np.ones((sFO_Px, 1)))[0] / volume

print('Final time:', final_time, 's')
print('Average dC:', final_average_dC, 'atm/m^3')

# Aggiungi i dati finali al file CSV
with open('data/Solution.csv', 'a') as f:
    f.write(f'{final_time},{final_average_dC}\n')

Current time: 0.0 s
Average dC: 0.0 atm/m^3

Current time: 10000.0 s
Average dC: 6.5401684790644144e+22 atm/m^3

Current time: 20000.0 s
Average dC: 1.276843428050113e+23 atm/m^3

Current time: 30000.0 s
Average dC: 1.8764979102545233e+23 atm/m^3

Current time: 40000.0 s
Average dC: 2.457074866699409e+23 atm/m^3

Current time: 50000.0 s
Average dC: 3.0211498772949227e+23 atm/m^3

Current time: 60000.0 s
Average dC: 3.570528153896797e+23 atm/m^3

Current time: 70000.0 s
Average dC: 4.106570232735897e+23 atm/m^3

Current time: 80000.0 s
Average dC: 4.630346792127817e+23 atm/m^3

Current time: 90000.0 s
Average dC: 5.14273034434467e+23 atm/m^3

Current time: 100000.0 s
Average dC: 5.6444434357204086e+23 atm/m^3

Current time: 110000.0 s
Average dC: 6.136112145618906e+23 atm/m^3

Current time: 120000.0 s
Average dC: 6.618258721393652e+23 atm/m^3

Current time: 130000.0 s
Average dC: 7.091362904816099e+23 atm/m^3

Current time: 140000.0 s
Average dC: 7.555857039217595e+23 atm/m^3

Current t

In [ ]:
print(sol_new_C.shape)  
np.savetxt('data/Concentration_field.csv', sol_new_C[:, :], delimiter=',', fmt='%d')
imported = np.loadtxt('Concentration_Field.csv', delimiter=',')
print(mass_Px.dot(imported[:, -1]).dot(np.ones((sFO_Px, 1)))[0] / volume)

(119763, 1001)
